<h1> "Momentum" Trading Stratergy </h1>
<ul>
    <li> Momentum strategy may be an ill-defined term and can mean a lot of different things. </li>
    <li> But what we will be looking for is a security or securities that are moving in a specific direction. </li>
        <ul>
            <li> We will screen and take into consideration securities that are above thier 50-day moving average. </li>
        </ul>
    <li> We can call this a <b> trend following stratergy </b>. </li>
        <ul>
            <li> We can make money as the security moves up and as it moves down. </li>
        </ul>
    <li> There is still research to be done on some trading signals that may be profitable before we start using them in the stratergy. </li>
    <li> Also, we would want to to generalize the signals to a number of securities. </li>
</ul>

In [24]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import yfinance as fin
from matplotlib import rcParams
import datetime as dt

<h3> 1. Download Data </h3>
<ul>
    <li> Typically trading "systems" involve a number of securities. </li>
    <li> In this stratergy I will be using GLD --> the gold ETF. </li>
</ul>

In [25]:
start_date = dt.datetime(2012,5,18)
end_date = dt.datetime(2023,1,10)
gold = fin.Ticker('GLD')
gold_hist = pd.DataFrame(gold.history(start = start_date, end = end_date))

In [26]:
gold_hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2012-05-18 00:00:00-04:00,154.460007,155.130005,153.970001,154.550003,12994200,0.0,0.0,0.0
2012-05-21 00:00:00-04:00,154.110001,154.949997,153.860001,154.649994,7542400,0.0,0.0,0.0
2012-05-22 00:00:00-04:00,153.589996,154.419998,151.550003,152.080002,11302700,0.0,0.0,0.0
2012-05-23 00:00:00-04:00,151.500000,151.839996,148.839996,151.619995,18854500,0.0,0.0,0.0
2012-05-24 00:00:00-04:00,152.710007,152.910004,150.619995,151.410004,9697100,0.0,0.0,0.0


In [27]:
Day = np.arange(1, len(gold_hist)+1)
gold_hist['Day'] = Day
gold_hist.drop(columns = ['Volume', 'Dividends', 'Stock Splits', 'Capital Gains'], inplace = True)
gold_hist = gold_hist[['Day', 'Open', 'High', 'Low', 'Close']]

In [28]:
gold_hist.head()

,Day,Open,High,Low,Close
Date,,,,,
2012-05-18 00:00:00-04:00,1,154.460007,155.130005,153.970001,154.550003
2012-05-21 00:00:00-04:00,2,154.110001,154.949997,153.860001,154.649994
2012-05-22 00:00:00-04:00,3,153.589996,154.419998,151.550003,152.080002
2012-05-23 00:00:00-04:00,4,151.500000,151.839996,148.839996,151.619995
2012-05-24 00:00:00-04:00,5,152.710007,152.910004,150.619995,151.410004


In [29]:
gold_hist.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2678 entries, 2012-05-18 00:00:00-04:00 to 2023-01-09 00:00:00-05:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Day     2678 non-null   int32  
 1   Open    2678 non-null   float64
 2   High    2678 non-null   float64
 3   Low     2678 non-null   float64
 4   Close   2678 non-null   float64
dtypes: float64(4), int32(1)
memory usage: 115.1 KB


<h3> 2. Add data/transform data </h3>
<ul>
    <li> Calculate signal based on some price or statistical acion </li>
    <li> I will be using a moving average crossover to generate signals </li>
    <li> For this stratergy I will always be "in" a trade, either long or short </li>
    <li> This is just a model; this means in real life variation should be expected </li>
</ul>

<h5> Add moving averages to the data frame </h5>